### Imports

In [48]:
# EDA common tools:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# preprocessing tools:
from sklearn.preprocessing import MinMaxScaler

# third-party pkg for stock:
import yfinance as yf

# ML - caveat for tf > 2.x, you need to proceed as follows:
import tensorflow as tf
from tensorflow.python import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

# print("TensorFlow version:", tf.__version__)
# print("All imports successful!")

### Download the desired stocks: APPL (Apple), and MSFT (Microsoft)

In [55]:
stocks = ['AAPL', 'MSFT']  # Example stock symbol
data = yf.download(stocks, start='2010-01-01', end='2023-01-01')
data = data['Close']

[*********************100%%**********************]  2 of 2 completed


#### Getting to know your data

In [56]:
data.head()

Ticker,AAPL,MSFT
Date,,
2010-01-04,7.643214,30.950001
2010-01-05,7.656429,30.959999
2010-01-06,7.534643,30.770000
2010-01-07,7.520714,30.450001
2010-01-08,7.570714,30.660000


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3272 entries, 2010-01-04 to 2022-12-30
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    3272 non-null   float64
 1   MSFT    3272 non-null   float64
dtypes: float64(2)
memory usage: 76.7 KB


In [58]:
# returns the amount of missing data:
data.isna().sum().sum()

0

### Preprocessing:

In [68]:
# scale the data so we can work with:
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)
scaled_data.head()

Ticker,AAPL,MSFT
Date,,
2010-01-04,0.004478,0.024805
2010-01-05,0.004553,0.024836
2010-01-06,0.003858,0.024242
2010-01-07,0.003778,0.023243
2010-01-08,0.004064,0.023899


In [69]:
# split the dataset:
prediction_days = 60

x_train = {}
y_train = {}

for stock in stocks:
    x_train[stock], y_train[stock] = [], []
    for x in range(prediction_days, len(scaled_data)):
        x_train[stock].append(scaled_data[stock].values[x-prediction_days:x])
        y_train[stock].append(scaled_data[stock].values[x])
    x_train[stock], y_train[stock] = np.array(x_train[stock]), np.array(y_train[stock])
    x_train[stock] = np.reshape(x_train[stock], (x_train[stock].shape[0], x_train[stock].shape[1], 1))

### Build the RNN model 

In [70]:
models = {}

for stock in stocks:
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train[stock].shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    models[stock] = model
    models[stock].summary()


/Users/leandro/miniconda3/envs/stock_prediction_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)